## Waiver Prediction Models


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

In [2]:
player_data = pd.read_csv("Data/merged_data/train_data.csv")

In [3]:
from sklearn.base import BaseEstimator, TransformerMixin
class YearWiseStandardScaler(BaseEstimator,TransformerMixin):
    def __init__(self, year_column):
        self.year_column = year_column
        self.year_stats = {}
    def fit(self, X, y = None):
        grouped = X.groupby(self.year_column)
        for year, group in grouped:
            self.year_stats[year] = {
                "mean": group.mean(),
                "std": group.std(ddof=0), 
            }
        return self
    def transform(self, X):
        def scale_row(row):
            year = row[self.year_column]
            stats = self.year_stats[year]
            return (row - stats["mean"]) / stats["std"]

        scaled_data = X.apply(scale_row, axis=1)
        scaled_data = scaled_data.drop(columns=self.year_column)
        
        return scaled_data

In [4]:
#Checking that my YearWiseStandardScaler works
toy_data = {
    "Feature1": [1.5, 1.0, 2.4, 0.0, 8.1, 9.2],
    "Feature2": [10, 20, 30, 40, 50, 60],
    "Year": [2023, 2022, 2022, 2023, 2019, 2022]
}

In [5]:
toy_df = df = pd.DataFrame(toy_data)

In [6]:
scaler = YearWiseStandardScaler(year_column ="Year")
scaler.fit(toy_df)
scaled_df = scaler.transform(toy_df)
print(scaled_df)

   Feature1  Feature2
0  1.000000 -1.000000
1 -0.893497 -0.980581
2 -0.502592 -0.392232
3 -1.000000  1.000000
4       NaN       NaN
5  1.396089  1.372813


In [7]:
len(player_data)

12655

In [8]:
player_data['WAIVED_BY_START_OF_NEXT_SEASON'] = player_data[['WAIVED', 'RELEASED']].any(axis=1).astype(int)

In [9]:
player_data.sample()

,NAME,PLAYER_ID,SEASON_START,TEAMS_LIST,PLAYER_AGE,EXPERIENCE,POS,GP,GS,MIN,...,WAIVED_REG,WAIVED_POST,RELEASED_OFF,RELEASED_REG,RELEASED_POST,TRADED_OFF,TRADED_REG,TRADED_POST,IN_LEAGUE_NEXT,WAIVED_BY_START_OF_NEXT_SEASON
7034,Evan Fournier,203095,2015,['ORL'],23.0,4,SF,79,71.0,2566.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0


In [10]:
player_data = player_data.dropna(subset=['SALARY'])

In [11]:
len(player_data)

12143

In [12]:
player_data = player_data[player_data['MIN'] != 0]

In [13]:
len(player_data)

12140

In [14]:
columns_to_normalize = ['FGM', 'FGA', 'PTS', 'PF', 'DREB', 'OREB', 'REB', 'FTA', 'FTM', 'STL', 'TOV', 'BLK', 'AST', 'FG3A', 'FG3M']

# Normalize the selected columns by dividing by 'MIN'
player_data[columns_to_normalize] = player_data[columns_to_normalize].div(player_data['MIN'], axis=0)

# Rename columns to include "per minute"
rename_dict = {col: f"{col} / MIN" for col in columns_to_normalize}
player_data.rename(columns=rename_dict, inplace=True)

In [15]:
player_data.sample()

,NAME,PLAYER_ID,SEASON_START,TEAMS_LIST,PLAYER_AGE,EXPERIENCE,POS,GP,GS,MIN,...,WAIVED_REG,WAIVED_POST,RELEASED_OFF,RELEASED_REG,RELEASED_POST,TRADED_OFF,TRADED_REG,TRADED_POST,IN_LEAGUE_NEXT,WAIVED_BY_START_OF_NEXT_SEASON
5723,Danilo Gallinari,201568,2020,['ATL'],32.0,12,PF,51,4.0,1222.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0


Now let's find the most correlated features

In [16]:
numeric_data = player_data.select_dtypes(include=['number'])

In [17]:
correlations = numeric_data.corr()['WAIVED_BY_START_OF_NEXT_SEASON']
sorted_correlations = correlations.sort_values(ascending=False)
pd.set_option('display.max_rows', None)

# Print the sorted correlations
print(sorted_correlations)

# Reset display options if needed
pd.reset_option('display.max_rows')

WAIVED_BY_START_OF_NEXT_SEASON    1.000000
WAIVED                            0.963348
WAIVED_REG                        0.765236
WAIVED_OFF                        0.621824
RELEASED                          0.264971
RELEASED_OFF                      0.235623
RELEASED_REG                      0.115759
PLAYER_ID                         0.107867
PF / MIN                          0.090211
SEASON_START                      0.089172
X3P_AR                            0.084789
WAIVED_POST                       0.078597
TOV_PERCENT                       0.070666
RELEASED_POST                     0.055563
PLAYER_AGE                        0.051323
FG3A / MIN                        0.038330
TRADED_REG                        0.031221
STL / MIN                         0.009669
TRADED                            0.009361
STL_PERCENT                       0.004476
OREB / MIN                       -0.003713
TOV / MIN                        -0.005039
ORB_PERCENT                      -0.008878
TRADED_OFF 

Since player_ID is correlated at 10% despite being complete random, let's take anything that's correlated at 15% or higher

In [18]:
Xfeatures = ['GP', 'MIN', 'DWS', 'GS', 'WS', 'OBPM', 'BPM', 'PER', 'OWS', 'WS_48', 'FT_PCT', 'FG_PCT', 'FGM / MIN', 'PTS / MIN', 'VORP', 'TS_PERCENT', 'SALARY', 'FTM / MIN', 'USG_PERCENT', 'FTA / MIN', 'FGA / MIN' , 'SEASON_START']
len(Xfeatures)

22

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
train_player_data, test_player_data = train_test_split(player_data, test_size=0.2, stratify=player_data['WAIVED_BY_START_OF_NEXT_SEASON'], random_state=812)
print(len(train_player_data))
print(len(test_player_data))

9712
2428


In [21]:
train_player_data_waived = train_player_data.loc[train_player_data['WAIVED_BY_START_OF_NEXT_SEASON'] == 1]
p = len(train_player_data_waived)/len(train_player_data)
print(p)
print(p*(1 - p))
bbs = p*(1 - p)

0.13807660626029655
0.1190114570639356


About 14% of the players are waived, so a randomly guessing waived or not with that probability would have an F1-score of about 14% and a Brier score of about .1175

In [22]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, brier_score_loss, balanced_accuracy_score

In [23]:

train_player_data = train_player_data.dropna(subset = Xfeatures)
print(len(train_player_data))
X = train_player_data[Xfeatures]
y = train_player_data['WAIVED_BY_START_OF_NEXT_SEASON']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=812, stratify = y)
knn_pipeline = Pipeline([
    ('scaler', YearWiseStandardScaler(year_column = 'SEASON_START')),
    ('knn', KNeighborsClassifier(n_neighbors = 15))
])
knn_pipeline.fit(X_train, y_train)
y_pred = knn_pipeline.predict(X_test)
f1score = f1_score(y_test, y_pred)
print("F1 score: ", f1score)
print("Balanced Accuracy: ", balanced_accuracy_score(y_test, y_pred))

9682
F1 score:  0.2411764705882353
Balanced Accuracy:  0.5671940798887936


In [24]:
y_prob = knn_pipeline.predict_proba(X_test)
y_prob = y_prob[:, 1]
brier_score = brier_score_loss(y_test, y_prob)
print("Brier Score: ", brier_score)
print("Brier Skill Score: ", 1 - brier_score/bbs)

Brier Score:  0.09675672575001436
Brier Skill Score:  0.18699654523148557


In [25]:
from sklearn.linear_model import LogisticRegression
X = train_player_data[Xfeatures]
y = train_player_data['WAIVED_BY_START_OF_NEXT_SEASON']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=813, stratify = y)


In [26]:
log_reg = Pipeline([
    ('scaler', YearWiseStandardScaler(year_column = 'SEASON_START')),
    ('logreg', LogisticRegression(max_iter=10000))
])

log_reg.fit(X_train, y_train)
y_pred = log_reg.predict(X_test)
f1score = f1_score(y_test, y_pred)
print("F1 score: ", f1score)
print("Balanced Accuracy: ", balanced_accuracy_score(y_test, y_pred))

F1 score:  0.27167630057803466
Balanced Accuracy:  0.5786009239952574


In [27]:
y_prob = log_reg.predict_proba(X_test)

In [28]:

y_prob
y_prob = y_prob[:, 1]
brier_score = brier_score_loss(y_test, y_prob)
print("Brier Score: ", brier_score)
print("Brier Skill Score: ", 1 - brier_score/bbs)

Brier Score:  0.09378056272941603
Brier Skill Score:  0.2120039108584728


In [29]:
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier

In [30]:
#tree_cfr = Pipeline([(scaler = YearWiseStandardScaler(year_column = 'SEASON_START'), (classifier = DecisionTreeClassifier(max_depth=6, random_state=814)))])
tree_cfr = Pipeline([
    ('scaler', YearWiseStandardScaler(year_column='SEASON_START')),  
    ('classifier', DecisionTreeClassifier(max_depth = 6, random_state=814))  
])
X = train_player_data[Xfeatures]
y = train_player_data['WAIVED_BY_START_OF_NEXT_SEASON']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=815, stratify = y)
tree_cfr.fit(X_train, y_train)
y_pred = tree_cfr.predict(X_test)
f1score = f1_score(y_test, y_pred)
print("F1 score: ", f1score)
print("Balanced Accuracy: ", balanced_accuracy_score(y_test, y_pred))

F1 score:  0.36228287841191065
Balanced Accuracy:  0.6187713225306749


In [31]:
y_prob = tree_cfr.predict_proba(X_test)
y_prob = y_prob[:, 1]
brier_score = brier_score_loss(y_test, y_prob)
print("Brier Score: ", brier_score)
print("Brier Skill Score: ", 1 - brier_score/bbs)

Brier Score:  0.0980026947731131
Brier Skill Score:  0.17652722526988407


In [32]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(
    n_estimators = 50, 
    max_depth = 3, 
    max_features = 6, 
    bootstrap= True, 
    max_samples = 500,
    random_state = 816
    )
rf = Pipeline([('scaler', YearWiseStandardScaler(year_column='SEASON_START')), ('classifier', RandomForestClassifier(
    n_estimators = 50, 
    max_depth = 3, 
    max_features = 6, 
    bootstrap= True, 
    max_samples = 500,
    random_state = 816
    ))])

In [33]:
X = train_player_data[Xfeatures]
y = train_player_data['WAIVED_BY_START_OF_NEXT_SEASON']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=817, stratify = y)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
f1score = f1_score(y_test, y_pred)
print("F1 score: ", f1score)
print("Balanced Accuracy: ", balanced_accuracy_score(y_test, y_pred))

F1 score:  0.22085889570552147
Balanced Accuracy:  0.560376548509751


In [34]:
y_prob = tree_cfr.predict_proba(X_test)
y_prob = y_prob[:, 1]
brier_score = brier_score_loss(y_test, y_prob)
print("Brier Score: ", brier_score)
print("Brier Skill Score: ", 1 - brier_score/bbs)

Brier Score:  0.0890786630266481
Brier Skill Score:  0.2515118693253787


Of the non-ensemble methods, the decision tree works best, so let's boost that one with AdaBoost

In [35]:
from sklearn.ensemble import AdaBoostClassifier

In [36]:
ada_clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=6),
                                n_estimators=100,
                                algorithm = 'SAMME',
                                learning_rate = 0.05,
                                random_state=818)
ada_clf = Pipeline([('scalar', YearWiseStandardScaler(year_column = 'SEASON_START')), ('classifier', AdaBoostClassifier(DecisionTreeClassifier(max_depth=6),
                                n_estimators=100,
                                algorithm = 'SAMME',
                                learning_rate = 0.05,
                                random_state=818))])

In [37]:
X = train_player_data[Xfeatures]
y = train_player_data['WAIVED_BY_START_OF_NEXT_SEASON']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=817, stratify = y)
ada_clf.fit(X_train, y_train)

Pipeline(steps=[('scalar', YearWiseStandardScaler(year_column='SEASON_START')),
                ('classifier',
                 AdaBoostClassifier(algorithm='SAMME',
                                    estimator=DecisionTreeClassifier(max_depth=6),
                                    learning_rate=0.05, n_estimators=100,
                                    random_state=818))])

In [38]:
y_pred = tree_cfr.predict(X_test)
f1score = f1_score(y_test, y_pred)
print("F1 score: ", f1score)
print("Balanced Accuracy: ", balanced_accuracy_score(y_test, y_pred))

F1 score:  0.3979591836734694
Balanced Accuracy:  0.6330559984736361


In [39]:
y_prob = ada_clf.predict_proba(X_test)
y_prob = y_prob[:, 1]
brier_score = brier_score_loss(y_test, y_prob)
print("Brier Score: ", brier_score)
print("Brier Skill Score: ", 1 - brier_score/bbs)

Brier Score:  0.09976458058416382
Brier Skill Score:  0.16172288748159702
